In [10]:
import pandas as pd 
import os

def merge_func(file_path, file_name):
    """
    지정된 경로에서 특정 파일 이름이 포함된 모든 Excel 파일을 찾아서 하나의 DataFrame으로 합칩니다.
    
    Parameters:
    file_path (str): 검색할 디렉토리 경로
    file_name (str): 찾을 파일 이름의 일부
    
    Returns:
    pd.DataFrame: 합쳐진 DataFrame
    """
    df_total = pd.DataFrame()
    
    try:
        # 디렉토리 내의 모든 파일 검색
        for file in os.listdir(file_path):
            # 파일이 Excel 파일이고 지정된 이름을 포함하는지 확인
            if (file.endswith('.xlsx') or file.endswith('.xls')) and file_name.lower() in file.lower():
                # 전체 파일 경로 생성
                full_path = os.path.join(file_path, file)
                
                try:
                    # Excel 파일 읽기
                    df_xlsx = pd.read_excel(io=full_path, header=2)
                    
                    # DataFrame 합치기
                    df_total = pd.concat([df_total, df_xlsx], axis=0, ignore_index=True)
                    
                except Exception as e:
                    print(f"파일 '{file}' 읽기 중 오류 발생: {str(e)}")
                    continue
        
        if df_total.empty:
            print(f"'{file_name}'이 포함된 Excel 파일을 찾을 수 없습니다.")
            
        return df_total
    
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return pd.DataFrame()

In [14]:

df_data = merge_func(f"../datasets/", f"주식거래 수수료").sort_values(by=["구분"],ascending=[True])

In [13]:
!pip install pymongo


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [18]:
from pymongo import MongoClient
# 네트워크 연결 규칙 : protocol://ip:port/path 
client = MongoClient('mongodb://192.168.0.91:27017/')

db_name = client["DB_TEST"]
col_target_name = db_name["COL_COMPARE_FEE"]

# df_data_without_id = df_data.drop(labels='_id', axis=1)
data_insert = df_data.to_dict(orient="records")
col_target_name.insert_many(data_insert)

InsertManyResult([ObjectId('67611223dde821a3a3d65747'), ObjectId('67611223dde821a3a3d65748'), ObjectId('67611223dde821a3a3d65749'), ObjectId('67611223dde821a3a3d6574a'), ObjectId('67611223dde821a3a3d6574b'), ObjectId('67611223dde821a3a3d6574c'), ObjectId('67611223dde821a3a3d6574d'), ObjectId('67611223dde821a3a3d6574e'), ObjectId('67611223dde821a3a3d6574f'), ObjectId('67611223dde821a3a3d65750'), ObjectId('67611223dde821a3a3d65751'), ObjectId('67611223dde821a3a3d65752'), ObjectId('67611223dde821a3a3d65753'), ObjectId('67611223dde821a3a3d65754'), ObjectId('67611223dde821a3a3d65755'), ObjectId('67611223dde821a3a3d65756'), ObjectId('67611223dde821a3a3d65757'), ObjectId('67611223dde821a3a3d65758'), ObjectId('67611223dde821a3a3d65759'), ObjectId('67611223dde821a3a3d6575a'), ObjectId('67611223dde821a3a3d6575b'), ObjectId('67611223dde821a3a3d6575c'), ObjectId('67611223dde821a3a3d6575d'), ObjectId('67611223dde821a3a3d6575e'), ObjectId('67611223dde821a3a3d6575f'), ObjectId('67611223dde821a3a3d657

In [22]:
data_cursor = col_target_name.find({'구분' : '변경후'})
data_list = list(data_cursor)
print(len(data_list))
df_comp_fee = pd.DataFrame(data_list)
df_comp_fee.head()

216


,_id,회사명,수수료\n부과기준,기준일자,거래금액,구분,오프라인,HTS,ARS,스마트폰,오프라인.1,HTS.1,ARS.1,스마트폰.1,비고
0,67611223dde821a3a3d658bd,DS투자증권,/img/sub/ico_hwp.gif,2021/01/01,100만원,변경후,4956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,67611223dde821a3a3d658be,엘에스증권,/img/sub/ico_pdf.gif,2012/12/03,50만원,변경후,2400.0,75.0,500.0,75.0,1500.0,75.0,500.0,75.0,거래금액 5천만원 미만 : 0.48%\n거래금액 5천~3억 미만 : 0.45%\n거...
2,67611223dde821a3a3d658bf,케이프투자증권,/img/sub/ico_doc.gif,2015/04/20,100만원,변경후,6000.0,3000.0,NaN,900.0,6000.0,150.0,NaN,150.0,1. 고객등급 실버등급은 당일 종목당 매수/매도 매체별 합산 하여 50만원 미만시 ...
3,67611223dde821a3a3d658c0,유진투자증권,/img/sub/ico_xls.gif,2020/06/17,500만원,변경후,25000.0,7500.0,10000.0,7500.0,25000.0,750.0,10000.0,750.0,NaN
4,67611223dde821a3a3d658c1,비엔피파리바증권,/img/sub/ico_pdf.gif,2020/02/03,50만원,변경후,1500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
total_mean = df_comp_fee["스마트폰.1"].dropna().mean()
total_mean

17731.059210526317

In [50]:
df_mean = df_comp_fee.dropna(subset=["스마트폰.1"]).groupby(by="회사명")["스마트폰.1"].mean()

df_mean.info()
df_mean

<class 'pandas.core.series.Series'>
Index: 26 entries, DB금융투자 to 현대차증권
Series name: 스마트폰.1
Non-Null Count  Dtype  
--------------  -----  
26 non-null     float64
dtypes: float64(1)
memory usage: 416.0+ bytes


회사명
DB금융투자      20904.166667
IBK투자증권     19433.333333
KB증권        23320.000000
NH투자증권      38565.500000
SK증권        24483.333333
교보증권        29241.666667
다올투자증권       2913.333333
대신증권        38341.166667
메리츠증권       19286.666667
미래에셋증권       2720.666667
부국증권        29066.666667
비엔케이투자증권    29150.000000
삼성증권        21250.333333
상상인증권       27206.666667
신영증권        19270.833333
아이엠증권       19389.166667
엘에스증권        2915.000000
유안타증권       19433.333333
유진투자증권       2915.000000
케이프투자증권      2915.000000
키움증권         2913.333333
하나증권          466.000000
한국투자증권       2731.000000
한양증권        19433.333333
한화투자증권      29150.000000
현대차증권        3698.000000
Name: 스마트폰.1, dtype: float64

In [52]:
df_mean_value = df_comp_fee.copy()
df_mean_value['avg'] = df_comp_fee['회사명'].map(df_mean)
df_fin = df_mean_value[['회사명','거래금액','스마트폰.1','avg']].sort_values(by=['회사명','거래금액','스마트폰.1','avg'],ascending=[True,True,True,True])

In [54]:
df_fin[df_fin['avg'] <= total_mean].to_csv("../datasets/Q_Compare_fee.csv")